In [75]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, sys, platform
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [76]:
print(os.getcwd())
import os
data = pd.read_csv(r'C:\Users\bpandey1\Downloads\ml\winequality-red.csv',sep=';')
data.info()

C:\Users\bpandey1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [77]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [78]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

RandomForest with default hyperparameters

In [79]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
rf_model = RandomForestRegressor(n_estimators=20,max_depth=10,min_samples_split=5,min_samples_leaf=5,max_features= None )
rf_model.fit(x_train,y_train)
y_pred = rf_model.predict(x_test)
mse_rf = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse_rf) 

Mean Squared Error: 0.345973774586943


In [80]:
def RF(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features):
    # Creating a Random Forest Regression Model and Fitting it to the Training Data
    rf_model = RandomForestRegressor(n_estimators=n_estimators,max_depth=max_depth,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,max_features= max_features)
    rf_model.fit(x_train,y_train)
    y_pred = rf_model.predict(x_test)
    score = r2_score(y_test, y_pred)
    return y_pred
    

In [81]:
# Define the hyperparameter grid
param_grid = {
    'n_estimators': (50, 400),
    'max_depth': (1, 30),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 4),
    'max_features': ['sqrt', 'log2', None]
}
 
 

model_rf = RandomForestRegressor()
# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model_rf, 
                                   param_distributions=param_grid, 
                                   n_iter=100, 
                                   cv=5, 
                                   random_state=42,
                                   n_jobs=-1)

# Perform the search
random_search.fit(x_train, y_train)

# Get the best parameters and best score
best_params = random_search.best_params_
best_score = random_search.best_score_
print("Best Hyperparameters:", best_params)


C:\Users\bpandey1\.conda\envs\pml\lib\site-packages\sklearn\model_selection\_search.py:307: UserWarning: The total space of parameters 48 is smaller than n_iter=100. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Hyperparameters: {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 30}


In [82]:
best_n_estimators = best_params[ 'n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']
best_max_features = best_params['max_features']

In [83]:
y_pred = RF(best_n_estimators, best_max_depth, best_min_samples_split, best_min_samples_leaf, best_max_features)
mse_rf = mean_squared_error(y_test, y_pred)

print("Randomized SearcCV for Random Fores algorithm:")
print("Mean Squared Error:", mse_rf)

Randomized SearcCV for Random Fores algorithm:
Mean Squared Error: 0.31560792968749996


Support Vector Machine using Randomized Search

In [85]:
from sklearn.svm import SVR
svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1, gamma='scale')
svr_model.fit(x_train, y_train)
y_pred_svr = svr_model.predict(x_test)
mse_svr = mean_squared_error(y_test, y_pred_svr)
print("Mean Squared Error (MSE):", mse_svr)

Mean Squared Error (MSE): 0.4742553073530834


In [22]:
#def SVR_hpo(C, epsilon, gamma, kernel, degree):
    # Creating a Random Forest Regression Model and Fitting it to the Training Data
    #svm_model = SVR(C=C, gamma=gamma, kernel=kernel)
   #svm_model.fit(x_train,y_train)
   # y_pred = svm_model.predict(x_test)
   # score = r2_score(y_test, y_pred)
   # return y_pred

In [88]:
param_grid = {
    'C': np.logspace(3, 100),
    'epsilon': np.logspace(3, 60),
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': np.arange(1, 10),
    'coef0': np.linspace(0, 15),
    'shrinking': [True, False]
}
svr_model = SVR()
random_search_svr = RandomizedSearchCV(estimator=svr_model,
                                       param_distributions=param_grid,
                                       n_iter=1,
                                       cv=5,
                                       random_state=42,
                                       n_jobs=-1)
# Perform the search
random_search_svr.fit(x_train, y_train)

# Get the best parameters and best score
best_params_svr = random_search_svr.best_params_
best_score_svr = random_search_svr.best_score_

print("Best Hyperparameters for SVR:", best_params_svr)

Best Hyperparameters for SVR: {'shrinking': True, 'kernel': 'rbf', 'gamma': 'auto', 'epsilon': 7906.043210907702, 'degree': 5, 'coef0': 1.0204081632653061, 'C': 37275937.20314938}


In [89]:
 # Using the best parameters to train the SVR model
best_svr = SVR(**best_params_svr)
best_svr.fit(x_train, y_train)
y_pred_svr = best_svr.predict(x_test)

# Evaluate the model
mse_svr = mean_squared_error(y_test, y_pred_svr)
mae_svr = mean_absolute_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)
print(" Using Best Parameters obtained from Hyperparameter optimization:")
print(" The Mean Squared Error:", mse_svr)
 

 Using Best Parameters obtained from Hyperparameter optimization:
 The Mean Squared Error: 0.58125


Bayesian Optimization for RandomForest Regression

In [90]:
pbounds = {'n_estimators': (50, 1000),
           'max_depth': (1, 30),
           'min_samples_split': (2, 10),
           'max_features':  ('sqrt', 'log2', None),
           'min_samples_leaf': (1, 4)}
rf_model = RandomForestRegressor()

optimizer = BayesSearchCV(
    estimator=rf_model,
    search_spaces=pbounds,
    n_iter=1,
    cv=5,
    n_jobs=-1
)

optimizer.fit(x_train, y_train)

# Get the best hyperparameters
best_params = optimizer.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: OrderedDict([('max_depth', 27), ('max_features', 'log2'), ('min_samples_leaf', 1), ('min_samples_split', 9), ('n_estimators', 777)])


In [91]:
# Train the model with the best hyperparameters
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']
best_max_features = best_params['max_features']
y_pred = RF(int(best_n_estimators), int(best_max_depth), int(best_min_samples_split), int(best_min_samples_leaf),best_max_features)

mse = mean_squared_error(y_test, y_pred)

print("HPO using Bayesian Optimization  for Random Forest algorithm:")
print("Mean Squared Error:", mse)

HPO using Bayesian Optimization  for Random Forest algorithm:
Mean Squared Error: 0.32348603062388126


 Bayesian Optimization For Support Vector Machine 

In [92]:
from sklearn.svm import SVR
from skopt import BayesSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Define the hyperparameter search space
pbounds = {
    'C': (1e-3, 1e3),
    'epsilon': (1e-3, 1e3),
    'gamma': (1e-6, 1e1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': (1, 6),
    'coef0': (0.0, 10.0),
    'shrinking': [True, False]
}

# Initialize SVR model
svr_model = SVR()

# Initialize BayesSearchCV
optimizer = BayesSearchCV(
    estimator=svr_model,
    search_spaces=pbounds,
    n_iter=1,
    cv=5, 
    n_jobs=-1
)

# Perform the search
optimizer.fit(x_train, y_train)

# Get the best hyperparameters
best_params = optimizer.best_params_
print("Best Hyperparameters:", best_params)

# Train the model with the best hyperparameters
best_svr = SVR(**best_params)
best_svr.fit(x_train, y_train)
y_pred = best_svr.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("HPO using Bayesian Optimization for Support Vector Machine algorithm:")
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R2 Score:", r2)


Best Hyperparameters: OrderedDict([('C', 301.85066224898054), ('coef0', 2.8749805200359195), ('degree', 1), ('epsilon', 376.72589518385706), ('gamma', 5.168141446685119), ('kernel', 'linear'), ('shrinking', True)])
HPO using Bayesian Optimization for Support Vector Machine algorithm:
Mean Squared Error: 0.58125
Mean Absolute Error: 0.65
R2 Score: -0.01535312180143289
